In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
import yfinance as yfin

import requests
import xlsxwriter
from datetime import date
import datetime

In [2]:
###@@@@ ADD .NS to stock names as yahoo uses [StockName].NS

def stockName(fileName):
    data=pd.read_csv(fileName)
    data.dropna()
    stocks=data['Symbol']
    MC=data['Market capitalization']  ## Market cap
    
    for i in range(0,len(stocks)):
        if (pd.isna(stocks[i])==False):
            stocks[i]=stocks[i] +'.NS'
#             if (i<len(stocks)-1):
#                 MC.append(data.iloc[i:i+1,3:4])
    return stocks,MC

In [3]:
################Get the sector list details for nifty500#################
dict_sector={}
dict_sector1=[]
df_sector=pd.read_csv('Nifty500_sector_list.csv')
df_sector
dict_sector={'Stock':df_sector['Symbol']+'.NS','Sector':df_sector['Industry']}
# dict_sector1={df_sector['Symbol'][0]:df_sector['Industry'][0]}
for k in range(0,len(df_sector)):
    dict_sector1.append({df_sector['Symbol'][k]+'.NS':df_sector['Industry'][k]})

In [4]:
dict_sector['Stock'][380]

'RELIANCE.NS'

In [5]:
### This function returns the data for a certain duration of time @@@@@@@@@@@@@
#################@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def companyData(name,duration,start,end):
    yfin
    
    # name = 'HDFCLIFE.NS'
    ticker = yfin.Ticker(name)
    #sector=ticker.info['sector']
    if (len(start)!= 0 and len(end)!=0):
        data=ticker.history(start=start,end=end)
    else:
        data=ticker.history(period=duration)
#         print(name)
#     sector=yfin.sector
    if (len(data)!=0):        
        month=data.index.month
        month=month.value_counts().index
        month=month.sort_values(ascending=True)
        year=data.index.year
        year= year.value_counts().index
        year=year.sort_values(ascending=True)
        return (data,ticker,year,month)
    else:
        print(ticker)
        return(np.NAN,np.NAN,np.NAN,np.NAN)

In [6]:
def percDiff(a,b):
    perc= ((b-a)*100)/a
    return perc

In [7]:
def retForFixedPeriod(data):
    if data is np.NAN:
        return np.NAN
    else:
        mnth_OP=[]  ##month open price
        mnth_CP=[]  ##month close price

        mnth_OP= data['Open'][0]
        mnth_CP= data['Close'][len(data)-1]
        ret=percDiff(mnth_OP,mnth_CP)
        return ret

In [8]:
def niftyReturn(stocks,MC):
    twoDayRet=[] ## 2 days ret
    week_ret=[] ## one week return
    month_ret=[] ##previous monthly return
    three_month_ret=[] ## 3 monthly return
    six_month_ret=[] ## 6 month return
    one_year_ret=[] ## 1 year return
    one_year_percentile=[] # 1 year precentile
    six_month_percentile=[]
    three_month_percentile=[]
    month_percentile=[]
    fiftytwo_week_high=[]
    fiftytwo_week_low=[]
    Avg_Score=[] ## average of all percentiles
    dict_nifty_returns={} ### dictionary containing the data on return percentage
    sector=[] ### sector of all the stocks
    for name in stocks:
        twoDayData,ticker,year,month=companyData(name,'2d','','')
        twoDayRet.append(retForFixedPeriod(twoDayData))
        OneWkData,ticker,year,month=companyData(name,'1wk','','')        
#         print(twoDayRet,'2dayret',OneWkData)
#         if twoDayRet is not np.NAN:
        week_ret.append(retForFixedPeriod(OneWkData))
        OneMonData,ticker,year,month=companyData(name,'1mo','','')
        month_ret.append(retForFixedPeriod(OneMonData))
        ThreeMonData,ticker,year,month=companyData(name,'3mo','','')
        three_month_ret.append(retForFixedPeriod(ThreeMonData))
        SixMonData,ticker,year,month=companyData(name,'6mo','','')
        six_month_ret.append(retForFixedPeriod(SixMonData))
        OneYrData,ticker,year,month=companyData(name,'1y','','')
        one_year_ret.append(retForFixedPeriod(OneYrData))
        
        if OneYrData is not np.NAN:
            fiftytwo_week_high.append(max(OneYrData))
            fiftytwo_week_low.append(min(OneYrData))
        else:
            fiftytwo_week_high.append(np.NAN)
            fiftytwo_week_low.append(np.NAN)
        
        
        for k in range(0,len(dict_sector1)):
            if dict_sector['Stock'][k]==name :
                sector.append(dict_sector['Sector'][k])
            else:
                sector.append('NaN')
    for i in range(0, len(stocks)):          
        x=stats.percentileofscore(month_ret,month_ret[i])/100
        y=stats.percentileofscore(three_month_ret,three_month_ret[i])/100
        z=stats.percentileofscore(six_month_ret,six_month_ret[i])/100
        n=stats.percentileofscore(one_year_ret,one_year_ret[i])/100
        month_percentile.append(x)
        three_month_percentile.append(y)
        six_month_percentile.append(z)
        one_year_percentile.append(n)
        Avg_Score.append((x+y+z+n)/4)
    dict_nifty_returns={'Name':stocks,'2DayRet':twoDayRet,'1WkRet':week_ret,'1MonRet':month_ret,'1MonPercentile':month_percentile,
      '3MonRet':three_month_ret,'3MonPercentile':three_month_percentile,
     '6Monet':six_month_ret,'6MonPercentile':six_month_percentile,
      '1YrRet':one_year_ret,'1YrPercentile':one_year_percentile,'AvgScore':Avg_Score, 'Market Cap':MC,'Sector':sector}
    
    return dict_nifty_returns

In [62]:
def supres(stocks):
    priceSup=[]  #support ratio
    priceRes=[]  ##resistance ratio
    min_price_supp_ratio=[] ## ratio for support
    min_price_res_ratio=[] ## ratio for resistance
    dict_priceSup={} ## dictionary for price support data
    yr_range=[] ## years for which data is obtained
    stocks_name=[]
    for name in stocks:
        data,ticker,year,month=companyData(name,'','2000-01-01','2022-01-01')
        dx=[]
        yr_range_temp=[]
        
        if data is not np.NAN:
            for m in range(0,len(year)):
                if m==0:
                    y=str(year[m])+'-01-01'
                    z=str(year[m]+2)+'-01-01'
                    x=year[m]+2
    #                 print(y,z)
                    yr_range_temp.append(year[m]+2)
                    dx.append(data[(data.index>y) & (data.index<z)])
                elif(x<max(year)):
                    #print(x)
                    y=str(x)+'-01-01'
                    z=str(x+2)+'-01-01'
                    x=x+2
    #                 print(y,z)
                    yr_range_temp.append(x)
                    dx.append(data[(data.index>y) & (data.index<z)])

            yr_range.append(yr_range_temp)
            dx.append(data[(data.index>str(x)+'-01-01')])
            CMP=data['High'][len(data)-1]
            lst_mx=[] ## store max values for a interval of 2 years over a period of 20 yrs
            lst_mn=[] ## store min values for a interval of 2 years over a period of 20 yrs
            lst1=[]

    #         print(name,end='\r')
            for k in range(0,len(dx)):
                if dx[k]['Close'].empty==False:
                    lst_mx.append(max(dx[k]['Close']))
                    lst_mn.append(min(dx[k]['Close']))
                    lst1.append({'Max':max(dx[k]['Close']),'Date':dx[k][(dx[k]['Close']==max(dx[k]['Close']))].index.year})

            ratio=[]
            ratio_min=[]
            for z in lst_mx:
                ratio.append(abs((z-CMP)/z))
            priceSup.append(ratio)
            min_price_supp_ratio.append(min(ratio))

            for z1 in lst_mn:
                ratio_min.append(abs((z1-CMP)/z1))
            priceRes.append(ratio_min)
            min_price_res_ratio.append(min(ratio_min))
            #print(name,priceRes)
            #print(min_price_res_ratio)
            dict_priceSup={'Name':stocks,'Support Ratio':priceSup,'Min Support Ratio':min_price_supp_ratio,
                               'Resistance Ratio':priceRes,"Min Resistance Ratio":min_price_res_ratio,"Year Range":yr_range}
            
#             return dict_priceSup 
        else:
            print(stocks.index(),'---ffff')
#             print(stocks[763],'df--',type(stocks))
            stocks.drop(stocks.loc[stocks.index==name].index)

    return dict_priceSup

In [63]:
# print(datetime.datetime.now())
# stocks,MC=stockName('NIFTY_ALL.csv')
# dict_nifty_returns=niftyReturn(stocks[1500:1919],MC[1500:1919])
# # dict_nifty_returns=niftyReturn(stocks[761:764],MC[761:7640])
# #dict_nifty_returns=niftyReturn(stocks[0:500],MC[0:500])
print(datetime.datetime.now())
# pd.DataFrame(dict_nifty_returns).sort_values(by='1WkRet',ascending=False)[20:40]
dict_priceSup=supres(stocks[1500:1919])
print(datetime.datetime.now())
# del dict_nifty_returns['Sector']
pd.DataFrame(dict_nifty_returns).sort_values(by='1WkRet',ascending=False)
print(dict_priceSup['Name'])
pd.DataFrame(dict_priceSup)
del dict_nifty_returns['Sector']
# print(dict_nifty_returns.keys())

2021-10-17 01:20:08.766404
- MHHL.NS: No data found, symbol may be delisted
yfinance.Ticker object <MHHL.NS>


TypeError: 'RangeIndex' object is not callable

In [22]:
print(dict_priceSup.keys())
print(len(dict_priceSup['Name']),len(dict_priceSup['Support Ratio']),len(dict_priceSup['Min Support Ratio']),len(dict_priceSup['Resistance Ratio']),len(dict_priceSup['Min Resistance Ratio']),len(dict_priceSup['Year Range']))

dict_keys(['Name', 'Support Ratio', 'Min Support Ratio', 'Resistance Ratio', 'Min Resistance Ratio', 'Year Range'])
499 497 497 497 497 497


In [46]:
dict_priceSup['Name']

501     SHILPAMED.NS
502     ORIENTREF.NS
503      POLYPLEX.NS
504     JAMNAAUTO.NS
505        JKTYRE.NS
           ...      
995    SASTASUNDR.NS
996            TI.NS
997          NDTV.NS
998        HLVLTD.NS
999           FEL.NS
Name: Symbol, Length: 499, dtype: object

In [12]:
df_100stocks_returns=pd.DataFrame(dict_nifty_returns)
df_100stocks_pricesup=pd.DataFrame(dict_priceSup)
df_100stocks=pd.merge(df_100stocks_returns,df_100stocks_pricesup)
df_100stocks[['Name','2DayRet','1MonRet','Support Ratio','Min Support Ratio']][0:25]
df_100stocks
# df_100stocks.sort_values(by='AvgScore',ascending=False)

NameError: name 'dict_nifty_returns' is not defined

In [18]:
##@@@@@ getting the fianancial rations and sorting based on PE,ROCE,ROE,D/E
def ratios(stocks,index1,index2):
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    import time

    ratios=[]
    ratio_values=[]
    dict_ratios={}
    MC=[]
    ROCE=[]
    ROE=[]
    PE=[]
    DE_Ratio=[]
    fiftyDMA=[]
    twohundredDMA=[]
    beta=[]
    EV=[]
    CMP=[]
    PB=[]
    stocks_NS=[]

    #data=pd.read_csv("NIFTY_ALL.csv")
    #data1=pd.read_csv("NIFTY_ALL.csv")
    #stocks=data['Symbol'][0:501]
    #stks=data1['Symbol'][0:501]
    stks=stocks
    print(len(stocks))
    for i in range(index1,index2):
#         stocks[i]=stocks[i]+'.NS'
#         time.sleep(2)
#         if(i==50 or i==100 or i==150 or i==200 or i==250 or i==300 or i==350 or i==400 or i==450):
#             print(i)
#             time.sleep(20)
        url=f"https://finance.yahoo.com/quote/{stocks[i]}/key-statistics?p={stocks[i]}"
        
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        soup.prettify()
        y=soup. find_all('span')
        x=soup.find_all('td')
        z=soup.find_all('tr')

        for l in range(0,len(z)):
            if (z[l].find_all('td')[0].get_text().find('Market Cap (intraday)')==0):
                if(z[l].find_all('td')[1].get_text().find('T')!=-1):
                    MC.append(float(z[l].find_all('td')[1].get_text().replace("T",""))*1000)
                elif(z[l].find_all('td')[1].get_text()!='N/A'):
                    MC.append(float(z[l].find_all('td')[1].get_text().replace("B","")))
                else:
                    MC.append(0)
            elif (z[l].find_all('td')[0].get_text().find('Enterprise Value 3')==0):
                if(z[l].find_all('td')[1].get_text().find('T')!=-1):
                    EV.append(float(z[l].find_all('td')[1].get_text().replace("T",""))*1000)
                elif(z[l].find_all('td')[1].get_text().find('B')!=-1):
                    EV.append(float(z[l].find_all('td')[1].get_text().replace("B","")))
                elif(z[l].find_all('td')[1].get_text().find('M')!=-1):
                    EV.append(float(z[l].find_all('td')[1].get_text().replace("M",""))/1000)
                else:
                    EV.append(0)
            elif (z[l].find_all('td')[0].get_text().find('50-Day Moving Average')==0):
                if(z[l].find_all('td')[1].get_text()!='N/A'):
                    fiftyDMA.append(float(z[l].find_all('td')[1].get_text().replace(",","")))
                else:
                    fiftyDMA.append(0)
            elif (z[l].find_all('td')[0].get_text().find('200-Day Moving Average')==0):
                if(z[l].find_all('td')[1].get_text()!='N/A'):
                    twohundredDMA.append(float(z[l].find_all('td')[1].get_text().replace(",","")))
                else:
                    twohundredDMA.append(0)
            elif (z[l].find_all('td')[0].get_text().find('Total Debt/Equity')==0):
                if(z[l].find_all('td')[1].get_text()=='N/A'):
                    DE_Ratio.append(0)
                else:
                    DE_Ratio.append(float(z[l].find_all('td')[1].get_text().replace(",","")))
            elif (z[l].find_all('td')[0].get_text().find('Beta')==0):
                if(z[l].find_all('td')[1].get_text()!='N/A'):
                    beta.append(float(z[l].find_all('td')[1].get_text().replace(",","")))
                else:
                    beta.append(0)
        stks[i]=stocks[i].replace(".NS","")
        print(stks[i],end="\r")
        url1=f"https://www.screener.in/company/{stks[i]}/consolidated"
        html1 = urlopen(url1)
        soup1 = BeautifulSoup(html1, 'lxml')
        y1=soup1.find_all('li',{'flex flex-space-between','default'})
        stk_temp=stks[i]
        for i in range(0,len(y1)):
        #### below code is added as sometime there is no data for consolidated in screener website###
            if(y1[i].find('span','name').get_text().split('\n')[2].strip()=='Current Price'):
                    if(len(y1[i].find('span','number').get_text().replace(",",""))==0):
                        url1=f"https://www.screener.in/company/{stk_temp}/"
                        html1 = urlopen(url1)
                        soup1 = BeautifulSoup(html1, 'lxml')
                        y1=soup1.find_all('li',{'flex flex-space-between','default'})
        ######End of code########################################################################
        
            if(y1[i].find('span','name').get_text().split('\n')[2].strip()=='ROCE'):
                if(len(y1[i].find('span','number').get_text().replace(",",""))==0):
                    roce=0
                else:
                    roce=float(y1[i].find('span','number').get_text().replace(",",""))
                #if(len(roce)==0):
                 #   roce='0'
                ROCE.append(roce)
            if(y1[i].find('span','name').get_text().split('\n')[2].strip()=='Current Price'):
                if(len(y1[i].find('span','number').get_text().replace(",",""))==0):
                    cmp=0
                else:
                    cmp=float(y1[i].find('span','number').get_text().replace(",",""))
                #if(len(cmp)==0):
                 #   cmp='0'
                CMP.append(cmp)
            if(y1[i].find('span','name').get_text().split('\n')[2].strip()=='Stock P/E'):
                if(len(y1[i].find('span','number').get_text().replace(",",""))==0):
                    pe=0
                else:
                    pe=float(y1[i].find('span','number').get_text().replace(",",""))
                #if(len(pe)==0):
                PE.append(pe)
            if(y1[i].find('span','name').get_text().split('\n')[2].strip()=='ROE'):
                if(len(y1[i].find('span','number').get_text().replace(",",""))==0):
                    roe=0
                else:
                    roe=float(y1[i].find('span','number').get_text().replace(",",""))
                #if(len(roe)==0):
                 #   roe='0'
                ROE.append(roe)
            if(y1[i].find('span','name').get_text().split('\n')[2].strip()=='Book Value'):
                if(len(y1[i].find('span','number').get_text().replace(",",""))==0):
                    PB.append(0)
                else:
                    pb=float(y1[i].find('span','number').get_text().replace(",",""))
                    PB.append(cmp/pb)
                #if(len(pb)==0):
                 #   pb='0'
                    #PB.append(np.nan)
                #else:
                 #   PB.append(cmp/pb)
                   
    dict_ratios={"Stock":stocks,"MC(billion INR)":MC,"P/E":PE,"CMP":CMP,"50DMA":fiftyDMA,"200DMA":twohundredDMA,"ROCE":ROCE,
                         "ROE":ROE,"debt/Eq":DE_Ratio,"PB":PB,"Beta":beta}
    #print(len(stocks),len(MC),len(PE),len(CMP),len(fiftyDMA),len(twohundredDMA),len(ROCE),len(ROE),len(DE_Ratio),len(PB),len(beta))
    return dict_ratios

In [98]:
print(datetime.datetime.now())
data=pd.read_csv("NIFTY_ALL.csv")
stocks=data['Symbol']
for i in range(501,1000):
    stocks[i]=stocks[i]+'.NS'
    
df_ratios1=pd.DataFrame(ratios(stocks[0:51],0,51))
df_ratios2=pd.DataFrame(ratios(stocks[51:101],51,101))
df_ratios3=pd.DataFrame(ratios(stocks[101:151],101,151))
df_ratios4=pd.DataFrame(ratios(stocks[151:201],151,201))
df_ratios5=pd.DataFrame(ratios(stocks[201:251],201,251))
df_ratios6=pd.DataFrame(ratios(stocks[251:301],251,301)) ###check this again
df_ratios7=pd.DataFrame(ratios(stocks[301:351],301,351))
df_ratios8=pd.DataFrame(ratios(stocks[351:401],351,401))
df_ratios9=pd.DataFrame(ratios(stocks[401:451],401,451))
df_ratios10=pd.DataFrame(ratios(stocks[451:501],451,501))
# df_ratios1=pd.DataFrame(ratios(stocks[501:551],501,551))
# df_ratios2=pd.DataFrame(ratios(stocks[551:601],551,601))
# df_ratios3=pd.DataFrame(ratios(stocks[601:651],601,651))
# df_ratios4=pd.DataFrame(ratios(stocks[651:701],651,701))
# df_ratios5=pd.DataFrame(ratios(stocks[701:751],701,751))
# df_ratios6=pd.DataFrame(ratios(stocks[751:801],751,801))
# df_ratios7=pd.DataFrame(ratios(stocks[801:851],801,851))
# df_ratios8=pd.DataFrame(ratios(stocks[851:901],851,901))
# df_ratios9=pd.DataFrame(ratios(stocks[901:951],901,951))
# df_ratios10=pd.DataFrame(ratios(stocks[951:1001],951,1001))
df_ratios=df_ratios1.append([df_ratios2,df_ratios3,df_ratios4,df_ratios5,df_ratios6,df_ratios7,df_ratios8,df_ratios9,df_ratios10])

df_ratios1
pe=[]
roce=[]
roe=[]
debt_eq=[]
x=[]
y=[]
z=[]
p=[]
score=[]

for m in range(0,len(df_ratios)):
    if(df_ratios['P/E'][m]==0):
        pe.append(0)
    else:
        pe.append(float(df_ratios['P/E'][m]))
    
    if(df_ratios['ROCE'][m]=='0'):
        roce.append(0)
    else:
        roce.append(float(df_ratios['ROCE'][m]))
        
    if(df_ratios['ROE'][m]=='0'):
        roe.append(0)
    else:
        roe.append(float(df_ratios['ROE'][m]))
        
    if(df_ratios['debt/Eq'][m]=='0' or df_ratios['debt/Eq'][m]=='N/A'):
        debt_eq.append(0)
    else:
#         print(m)
        #debt_eq.append(float(df_ratios['debt/Eq'][m].replace(",","")))
        debt_eq.append(df_ratios['debt/Eq'][m])
        
for k in range(0,len(df_ratios)):
    x.append(100-stats.percentileofscore(pe,pe[k])) ###since we need co.s with low PE we need to subtract from 100 as it gives the true picture
    y.append(stats.percentileofscore(roce,roce[k]))
    z.append(stats.percentileofscore(roe,roe[k]))
    p.append(100-stats.percentileofscore(debt_eq,debt_eq[k])) ###since we need co.s with low PE we need to subtract from 100 as it gives the true picture
    score.append((x[k]+y[k]+z[k]+p[k])/4)
df_ratios['PE percentile']=x
df_ratios['ROCE percentile']=y
df_ratios['ROE Percentile']=z
df_ratios['Debt/Eq percentile']=p
df_ratios['Avg Score']=score
print(datetime.datetime.now())
df_ratios


2021-06-21 21:44:56.452359
51


ValueError: could not convert string to float: '625.59M'

In [13]:
exportToExcel(df_100stocks,'NiftyReturns500.xlsx','Returns_NSE500')
# exportToExcel(df_ratios,'Nifty100_ratios.xlsx','Ratios_NSE500')

In [12]:
def exportToExcel(df,fileName,sheetname):
    
#     import openpyxl
#     from pandas import ExcelWriter
#     with ExcelWriter(fileName, mode='a') as writer:
#         df.to_excel(writer, sheet_name=sheetname,index=False)

#     worksheet1 = workbook.add_worksheet()
#     worksheet2 = workbook.add_worksheet()
    
#     xlsxwriter.load(fileName)
    writer = pd.ExcelWriter(fileName,engine='xlsxwriter')
    df.to_excel(writer, sheet_name=sheetname, index = False)
    
    wb = xlsxwriter.Workbook(fileName)
    
#     with pd.ExcelWriter(fileName,
#                     mode='a') as writer:  
#     df.to_excel(writer, sheet_name=sheetname)
#     df1.to_excel('output1.xlsx', engine='xlsxwriter')  
    
    background_color = '#0a0a23'
    font_color = '#ffffff'

    string_format = writer.book.add_format(
            {
                'font_color': font_color,
                'bg_color': background_color,
                'border': 1
            }
        )

    dollar_format = writer.book.add_format(
            {
                'num_format':'$0.00',
                'font_color': font_color,
                'bg_color': background_color,
                'border': 1
            }
        )

    integer_format = writer.book.add_format(
            {
                'num_format':'0',
                'font_color': font_color,
                'bg_color': background_color,
                'border': 1
            }
        )
    
    decimal_format = writer.book.add_format(
            {
                'num_format':'0.00',
                'font_color': font_color,
                'bg_color': background_color,
                'border': 1
            }
        )
#     dict_delta={'Stocks':stocks,'50DMA':fiftyMovAvg,'Sector':SECTOR,'3 Red CS':three_red, '2 Green CS':two_green,'Total Change':tot_chg}
#     column_formats={
#                         'A': ['Ticker', string_format],
#                         'B':['Market Cap',decimal_format],
#                         'C':['P/E',decimal_format],
#                         'D':['CMP',decimal_format],
#                         'E':['50DMA',decimal_format],
#                         'F':['200DMA',decimal_format],
#                         'G':['ROCE',decimal_format],
#                         'H':['ROE',decimal_format],
#                         'I':['debt/Eq',decimal_format],
#                         'J':['PB',decimal_format],
#                         'K':['Beta',decimal_format],
#                         'L':['PE percentile',decimal_format],
#                         'M':['ROCE percentile',decimal_format],
#                         'N':['ROE Percentile',decimal_format],
#                         'O':['Debt/Eq percentile',decimal_format],
#                         'P':['Avg Score',decimal_format]
#                       }
    column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['2 Day Return',decimal_format],
                    'C': ['1 Week Return',decimal_format],
                    'D': ['1 Month Return', decimal_format],
                    'E': ['1 Month Percentile', decimal_format],
                    'F': ['3 Month Return',decimal_format],
                    'G': ['3 Month Percentile', decimal_format],
                    'H': ['6 Month Return',decimal_format],
                    'I': ['6 Month Percentilr',decimal_format],
                    'J': ['1 Year Return',decimal_format],
                    'K': ['1 Year Percentile',decimal_format],
                    'L': ['Average Score',decimal_format],
                    'M': ['Market Cap',decimal_format],
                    'N': ['Support Ratio',decimal_format],
                    'O': ['Min Support Ratio',decimal_format],
                    'P': ['Resistance Ratio',decimal_format],
                    'Q': ['Min Resistance Ratio',decimal_format],
                    'R': ['Year Range',decimal_format]
                    }

    for column in column_formats.keys():
        writer.sheets[sheetname].set_column(f'{column}:{column}', 20, column_formats[column][1])
        writer.sheets[sheetname].write(f'{column}1', column_formats[column][0], string_format)

    writer.save()

In [ ]:
dict1={'a':['A',1],'b':['B',2],'c':['C',3],'d':['D',4]}
dfx=pd.DataFrame(dict1)
dfx.iloc[:,:4]

In [ ]:
dfx